In [1]:
import pyproj

# Converting between vertical datums

SeaBee pilots need to be able to convert between vertical datums as well as perform 2D co-ordinate conversions. This notebook explores options using [PyProj](https://pyproj4.github.io/pyproj/stable/index.html).

## Background

CRSs may be 2D or 3D. 3D systems use a 2D CRS for (x, y), plus an additional axis for elevations. For example, [EPSG 4326](https://epsg.org/crs_4326/WGS-84.html) is a 2D geographic CRS measured relative to the WGS84 ellipsoid, whereas [EPSG 4979](https://epsg.org/crs_4979/WGS-84.html) is the 3D equivalent, where x and y are longitude and latitude (just like for EPSG 4326) and z is elevation in metres measured relative to the WGS84 ellipsoid. 

It is also possible to have projected co-ordinates for x and y, with elevations measured relative to various underlying height models. For example, [EPSG 25833](https://epsg.org/crs_25833/ETRS89-UTM-zone-33N.html) is a 2D projected system (UTM Zone 33N) relative to the ETRS89 datum, while [EPSG 5973](https://epsg.org/crs_5973/ETRS89-UTM-zone-33N-NN2000-height.html) is a 3D compound CRS combining ETRS89 UTM Zone 33N with the NN2000 height model (which is the most up-to-date height model for Norway).

[This](https://register.geonorge.no/epsg-koder) website provides EPSG codes for various CRSs relevant to Norway. [This](https://gis.stackexchange.com/a/385738/2131) post from Stack Exchange is also helpful.

## Proj data

By default, Proj only installs the most commonly used transformation files. Sometimes it gives helpful errors when files are missing, but in some cases it fails "silently" i.e. the input co-ordinates are returned unchanged, but it otherwise everything seems to work. This problem can be solved by installing the necessary data files or, if disk space is not an issue, just installing everything using 

    mamba install -c conda-forge proj-data

In [2]:
# Test co-ords for near Larvik
lng = 10.13157
lat = 59.00788
z = 0

# WGS84 for horizontal and vertical components
in_crs = pyproj.crs.CRS.from_epsg(4979)

# ETRS89 UTM Zone 33N for horizontal and NN2000 for vertical
out_crs = pyproj.crs.CRS.from_epsg(5973)

In [3]:
in_crs

<Geographic 3D CRS: EPSG:4979>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- name: World: Afghanistan, Albania, Algeria, American Samoa, Andorra, Angola, Anguilla, Antarctica, Antigua and Barbuda, Argentina, Armenia, Aruba, Australia, Austria, Azerbaijan, Bahamas, Bahrain, Bangladesh, Barbados, Belgium, Belgium, Belize, Benin, Bermuda, Bhutan, Bolivia, Bonaire, Saint Eustasius and Saba, Bosnia and Herzegovina, Botswana, Bouvet Island, Brazil, British Indian Ocean Territory, British Virgin Islands, Brunei Darussalam, Bulgaria, Burkina Faso, Burundi, Cambodia, Cameroon, Canada, Cape Verde, Cayman Islands, Central African Republic, Chad, Chile, China, Christmas Island, Cocos (Keeling) Islands, Comoros, Congo, Cook Islands, Costa Rica, Côte d'Ivoire (Ivory Coast), Croatia, Cuba, Curacao, Cyprus, Czechia, Denmark, Djibouti, Dominica, Dominican Republic, East Timor, Ec

In [4]:
out_crs

<Compound CRS: EPSG:5973>
Name: ETRS89 / UTM zone 33N + NN2000 height
Axis Info [cartesian|vertical]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
- H[up]: Gravity-related height (metre)
Area of Use:
- name: Norway - onshore - between 12°E and 18°E.
- bounds: (12.0, 59.88, 18.01, 69.68)
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich
Sub CRS:
- ETRS89 / UTM zone 33N
- NN2000 height

In [5]:
# Build transformer
tform = pyproj.transformer.Transformer.from_crs(
    crs_from=in_crs, crs_to=out_crs, always_xy=True
)
tform.transform(lng, lat, z)

(220519.58047844184, 6551117.485868912, -39.71352391052247)

In other words, the WGS84 ellipsoid is 39.7 m above the surface of NN2000 in the area around Larvik. 